In [1]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [5]:
import pandas as pd
from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_p009.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(20650, 2552)
(20650, 7)


4h_williams_r_lag9                2611
4h_phase_accumulation_dt_lag10    2611
4h_fisher_lag4                    2611
4h_fisher_lag5                    2611
4h_fisher_lag6                    2611
                                  ... 
p009_stc                             0
p009_adaptive_rsi                    0
p009_comb_spectrum_pwr_10            0
p009_adaptive_bp                     0
p009_ac_29                           0
Length: 2552, dtype: int64

In [6]:
df_features = df_features[2611:]
label_side = label_side[2611:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(18039, 2552)
(18039, 7)


p0045_acc_swing_index_ddt    0
15m_williams_r_lag12         0
15m_williams_r_ddt           0
15m_williams_r_dt            0
15m_williams_r_dt_lag1       0
                            ..
p009_hurst_coef_slow_lag4    0
p009_hurst_coef_slow_lag5    0
p009_hurst_coef_slow_lag6    0
p009_hurst_coef_slow_lag7    0
4h_williams_r_lag9           0
Length: 2552, dtype: int64

In [7]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(18039, 2552)
(array([0, 1]), array([8496, 9543]))
(array([0, 1]), array([1632, 1976]))


In [9]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=150, n_jobs=1)

[I 2025-03-24 13:22:57,361] A new study created in memory with name: no-name-8e531f64-507f-42f0-ab06-a234db879604
[I 2025-03-24 13:23:01,270] Trial 0 finished with value: 0.8756865473922362 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 286, 'max_depth': 983, 'min_gain_to_split': 0.1352658276997148, 'min_data_in_leaf': 442, 'lambda_l1': 81.29196408460004, 'lambda_l2': 46.221726842321566, 'num_boost_round': 402}. Best is trial 0 with value: 0.8756865473922362.


precision: 0.8044806517311609, recall: 0.7995951417004049, f1: 0.8020304568527918


[I 2025-03-24 13:23:18,670] Trial 1 finished with value: 0.8752806347741526 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 259, 'max_depth': 284, 'min_gain_to_split': 0.6947165307397023, 'min_data_in_leaf': 440, 'lambda_l1': 77.27944681551892, 'lambda_l2': 5.919217581389301, 'num_boost_round': 1380}. Best is trial 0 with value: 0.8756865473922362.


precision: 0.8063040162684291, recall: 0.8026315789473685, f1: 0.8044636063910728


[I 2025-03-24 13:23:29,287] Trial 2 finished with value: 0.8753674610026196 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 277, 'max_depth': 499, 'min_gain_to_split': 0.7808203760977118, 'min_data_in_leaf': 345, 'lambda_l1': 71.42526075833929, 'lambda_l2': 38.89368778610693, 'num_boost_round': 1880}. Best is trial 0 with value: 0.8756865473922362.


...

precision: 0.8039014373716632, recall: 0.7925101214574899, f1: 0.7981651376146789


[I 2025-03-24 14:05:48,238] Trial 198 finished with value: 0.8773905121259029 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 199, 'max_depth': 942, 'min_gain_to_split': 0.8258749590602927, 'min_data_in_leaf': 73, 'lambda_l1': 37.69223191710954, 'lambda_l2': 74.16691931185538, 'num_boost_round': 1485}. Best is trial 131 with value: 0.8782798607803445.


precision: 0.8026859504132231, recall: 0.7864372469635628, f1: 0.7944785276073619


[I 2025-03-24 14:05:56,585] Trial 199 finished with value: 0.8778956547193777 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 151, 'max_depth': 925, 'min_gain_to_split': 0.8979199733881995, 'min_data_in_leaf': 101, 'lambda_l1': 40.4445269374483, 'lambda_l2': 69.7206529058998, 'num_boost_round': 1381}. Best is trial 131 with value: 0.8782798607803445.


precision: 0.8097713097713097, recall: 0.7884615384615384, f1: 0.798974358974359


In [11]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 205,
 'max_depth': 942,
 'min_gain_to_split': 0.9173035124799338,
 'min_data_in_leaf': 82,
 'lambda_l1': 36.011940679265564,
 'lambda_l2': 59.50815243888857,
 'num_boost_round': 1424}

In [12]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [13]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [14]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8782798607803445


In [15]:
model.save_model("custom_indicators/models/model_side.txt")
prod_model.save_model("custom_indicators/models/model_side_prod.txt")

# meta model

In [5]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_p009.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag18    2956
4h_dft_dom_cycle_lag19        2945
4h_dft_dom_cycle_dt_lag18     2945
4h_dft_dom_cycle_lag18        2933
4h_dft_dom_cycle_lag17        2902
                              ... 
p009_conv_25                     0
p009_conv_24                     0
p009_conv_23                     0
p009_conv_22                     0
side_model_res                   0
Length: 2552, dtype: int64

In [6]:
df_features = df_features[2959:]
meta_label = meta_label[2959:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(17691, 2552)
(17691, 7)


p0045_ac_13                             0
1h_comb_spectrum_dom_cycle_ddt_lag10    0
1h_bandpass_lag1                        0
1h_bandpass_lag10                       0
1h_bandpass_lag11                       0
                                       ..
p009_vwap_lag14                         0
p009_vwap_lag15                         0
p009_vwap_lag16                         0
p009_vwap_lag17                         0
side_model_res                          0
Length: 2552, dtype: int64

In [7]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(17691, 2552)
(array([0, 1]), array([ 2498, 11654]))
(array([0, 1]), array([ 778, 2761]))


In [10]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.3, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=150, n_jobs=1)

[I 2025-03-24 15:01:39,549] A new study created in memory with name: no-name-6a2bdcc1-134d-4eb5-a5c3-19cec8402a91
[I 2025-03-24 15:01:45,893] Trial 0 finished with value: 0.6226154299768762 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 449, 'max_depth': 724, 'min_gain_to_split': 0.5950963815957815, 'min_data_in_leaf': 210, 'lambda_l1': 44.58464836934562, 'lambda_l2': 25.60918019866989, 'num_boost_round': 488}. Best is trial 0 with value: 0.6226154299768762.


precision: 0.956266318537859, recall: 0.5306048533140166, f1: 0.6825064057768461, auc: 0.785911274276579, fbeta: 0.6226154299768762


[I 2025-03-24 15:01:57,111] Trial 1 finished with value: 0.726599636713455 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 102, 'max_depth': 125, 'min_gain_to_split': 0.022435666072832903, 'min_data_in_leaf': 258, 'lambda_l1': 67.83550457623424, 'lambda_l2': 3.067657747868412, 'num_boost_round': 502}. Best is trial 1 with value: 0.726599636713455.


precision: 0.8926807364167041, recall: 0.7200289750090547, f1: 0.7971130713712911, auc: 0.7857543883824366, fbeta: 0.726599636713455


[I 2025-03-24 15:02:19,059] Trial 2 finished with value: 0.5454090455029167 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 296, 'max_depth': 477, 'min_gain_to_split': 0.35174876188252935, 'min_data_in_leaf': 148, 'lambda_l1': 86.60526878781248, 'lambda_l2': 84.24987230333372, 'num_boost_round': 1973}. Best is trial 1 with value: 0.726599636713455.


...

precision: 0.8284923928077456, recall: 0.8678015211879754, f1: 0.8476914912435874, auc: 0.7739399960336267, fbeta: 0.7502972569487769


[I 2025-03-24 15:33:14,866] Trial 98 finished with value: 0.7485849452708507 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 147, 'max_depth': 73, 'min_gain_to_split': 0.09659025186121956, 'min_data_in_leaf': 292, 'lambda_l1': 16.188741887931208, 'lambda_l2': 40.11732920476551, 'num_boost_round': 1487}. Best is trial 96 with value: 0.7627559923489124.


precision: 0.8366247755834829, recall: 0.8438971387178559, f1: 0.8402452217814641, auc: 0.7775949252766918, fbeta: 0.7485849452708507


[I 2025-03-24 15:33:33,151] Trial 99 finished with value: 0.7613608669742641 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 96, 'max_depth': 169, 'min_gain_to_split': 0.12417331016717466, 'min_data_in_leaf': 246, 'lambda_l1': 5.705467362367361, 'lambda_l2': 31.07413439138597, 'num_boost_round': 1279}. Best is trial 96 with value: 0.7627559923489124.


precision: 0.8314606741573034, recall: 0.8844621513944223, f1: 0.8571428571428571, auc: 0.7759548392082523, fbeta: 0.7613608669742641


In [12]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 146,
 'max_depth': 70,
 'min_gain_to_split': 0.09747887558586912,
 'min_data_in_leaf': 283,
 'lambda_l1': 6.204654069068917,
 'lambda_l2': 29.895773680862263,
 'num_boost_round': 1397}

In [13]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [14]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [15]:
model.save_model("custom_indicators/models/model_meta.txt")
prod_model.save_model("custom_indicators/models/model_meta_prod.txt")